# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World


## Instructions

Total Points: Undergrads 100 / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a PDF file. 

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [311]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [312]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'east'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a performance measure which is here the number of cleaned squares (since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room as required in the PEAS description above). The energy budget of the agent is specified as `max_steps`. 

In [313]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned

Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [314]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: east
step 1 - action: west
step 2 - action: south
step 3 - action: south
step 4 - action: east
step 5 - action: north
step 6 - action: suck
step 7 - action: south
step 8 - action: west
step 9 - action: suck
step 10 - action: east
step 11 - action: east
step 12 - action: north
step 13 - action: east
step 14 - action: west
step 15 - action: south
step 16 - action: west
step 17 - action: east
step 18 - action: west
step 19 - action: west


2

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement inteligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formated professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. Use the same environmnt for all your agent implementations in the tasks below.

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [315]:
# Your code and description goes here

import random

#You can edit the size (assume square), number of dirty squares, max steps, tuple for position, and verboseness
def simple_environment(agent, enviro_size, num_dirty, max_steps, start_pos, verbose):
    num_cleaned = 0

    #Calculate the environment area, random of number squares dirty, and a range to assign values
    envi_area = enviro_size * enviro_size
    dirty_array = random.sample(range(envi_area), num_dirty)
    
    #Create the 2D Array with 'D' for dirty and 'C' for clean squares
    environment = []
    linIndex = 0
    for i in range(enviro_size):
        envi_row = []
        for j in range(enviro_size):
            if linIndex in dirty_array:
                envi_row.append('D')
            else:
                envi_row.append('C')
            linIndex += 1

        environment.append(envi_row)

    'Start with the position given and set default values'
    currX = start_pos[0]
    currY = start_pos[1]
    dirty = False
    bumpers = {"north" : True, "south" : True, "west" : True, "east" : True}

    'For loop for the amount of steps given'
    for steps in range(max_steps):
        #Checks the current location and sets the bumpers to correct values
        #If bumpers at value == False, there is a barrier. If it is True, it can move in that direction
        if currX == 0:
            bumpers["west"] = False
        else:
            bumpers["west"] = True

        if currX == enviro_size - 1:
            bumpers["east"] = False
        else:
            bumpers["east"] = True

        if currY == 0:
            bumpers["north"] = False
        else:
            bumpers["north"] = True

        if currY == enviro_size - 1:
            bumpers["south"] = False
        else:
            bumpers["south"] = True
        
        #Setting environment square state to dirty or clean
        if environment[currX][currY] == 'D':
            dirty = True
        else:
            dirty = False

        while True:
            action = agent(bumpers, dirty)
            if action == "north":
                if currY <= 0:
                    continue
                else:
                    currY -= 1
                    break
            if action == "south":
                if currY >= enviro_size - 1:
                    continue
                else:
                    currY += 1
                    break
            if action == "east":
                if currX >= enviro_size - 1:
                    continue
                else:
                    currX += 1
                    break
            if action == "west":
                if currX <= 0:
                    continue
                else:
                    currX -= 1
                    break
            if action == "suck":
                environment[currX][currY] = 'C'
                num_cleaned = num_cleaned + 1
                break
            if action == "ALL CLEAN!":
                return num_cleaned
                break
        
        #Reduce steps
        max_steps = max_steps - 1

        #Add some printing if verbose needed
        if verbose:
            print("Step", steps , "- action:", action) 
            print("Bumper Sensors:", bumpers)
            print("Current Location:", currX, ",", currY)
            print("Current Environment:", environment, '\n')
    if verbose: 
        print("Total Dirty:", num_dirty)
    
    return num_cleaned


## Task 2:  Implement a simple reflex agent [10 Points] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function.

In [316]:
# Your code and description goes here
def simple_reflex_agent(bumpers, dirty):
    #Suck ONLY if dirty
    if(dirty == True):
        return 'suck'
    else:
        #Pick a random direction until it returns true, then move there
        movements = ["north", "east", "west", "south"]
        pick = True
        while pick:
            thisMove = np.random.choice(movements)
            if(bumpers[thisMove] == True):
                return thisMove

#Print the output via the environment   
print("Total Cleaned:", simple_environment(simple_reflex_agent, 5, 10, 50, (0,0), True))

Step 0 - action: east
Bumper Sensors: {'north': False, 'south': True, 'west': False, 'east': True}
Current Location: 1 , 0
Current Environment: [['C', 'D', 'D', 'C', 'C'], ['C', 'C', 'D', 'C', 'D'], ['C', 'D', 'C', 'D', 'C'], ['D', 'C', 'C', 'D', 'C'], ['C', 'C', 'D', 'D', 'C']] 

Step 1 - action: west
Bumper Sensors: {'north': False, 'south': True, 'west': True, 'east': True}
Current Location: 0 , 0
Current Environment: [['C', 'D', 'D', 'C', 'C'], ['C', 'C', 'D', 'C', 'D'], ['C', 'D', 'C', 'D', 'C'], ['D', 'C', 'C', 'D', 'C'], ['C', 'C', 'D', 'D', 'C']] 

Step 2 - action: south
Bumper Sensors: {'north': False, 'south': True, 'west': False, 'east': True}
Current Location: 0 , 1
Current Environment: [['C', 'D', 'D', 'C', 'C'], ['C', 'C', 'D', 'C', 'D'], ['C', 'D', 'C', 'D', 'C'], ['D', 'C', 'C', 'D', 'C'], ['C', 'C', 'D', 'D', 'C']] 

Step 3 - action: suck
Bumper Sensors: {'north': True, 'south': True, 'west': False, 'east': True}
Current Location: 0 , 1
Current Environment: [['C', 'C',

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb))

#### Your short description of the state and your implementation goes here:
In this case, my state is represented by the variables stored inside the class of the agent. Here, I store whether the robot has been to the origin (0,0), it's current X and Y, and which direction it's heading (whether or not it's going East).

Using this, the agent knows its location and uses that to systematically clean the square room row by row.

In [317]:
# Your code goes here
class model_based_reflex_agent():
    def __init__(self):
        self.isOrigin = False
        self.currX = 0
        self.currY = 0
        self.goEast = True

    def goToOrigin(self, bumpers):
        if bumpers['north'] == False and bumpers['west'] == False:
            self.isOrigin = True
        else:
            if(bumpers['north'] != False):
                return 'north'
            if(bumpers['west'] != False):
                return 'west'

    def act(self, bumpers, dirty):
        if self.isOrigin == False:
            return self.goToOrigin(bumpers)
        else:
            if dirty == True:
                return 'suck'
            if bumpers['south'] == False and bumpers['east'] == False:
                return 'ALL CLEAN!'
            if self.goEast:
                if bumpers['east'] == True:
                    self.currX += 1
                    return 'east'
                if bumpers['east'] == False:
                    self.currY += 1
                    self.goEast = False
                    return 'south'
            else:
                if bumpers['west'] == True:
                    self.currX -= 1
                    return 'west'
                if bumpers['west'] == False:
                    self.currY += 1
                    self.goEast = True
                    return 'south'
             
        
agent = model_based_reflex_agent();       
print("Total Cleaned:", simple_environment(agent.act, 5, 10, 50, (3,2), True))

Step 0 - action: north
Bumper Sensors: {'north': True, 'south': True, 'west': True, 'east': True}
Current Location: 3 , 1
Current Environment: [['C', 'C', 'D', 'D', 'D'], ['C', 'C', 'D', 'C', 'D'], ['C', 'D', 'D', 'C', 'C'], ['C', 'C', 'C', 'D', 'D'], ['D', 'C', 'C', 'C', 'C']] 

Step 1 - action: north
Bumper Sensors: {'north': True, 'south': True, 'west': True, 'east': True}
Current Location: 3 , 0
Current Environment: [['C', 'C', 'D', 'D', 'D'], ['C', 'C', 'D', 'C', 'D'], ['C', 'D', 'D', 'C', 'C'], ['C', 'C', 'C', 'D', 'D'], ['D', 'C', 'C', 'C', 'C']] 

Step 2 - action: west
Bumper Sensors: {'north': False, 'south': True, 'west': True, 'east': True}
Current Location: 2 , 0
Current Environment: [['C', 'C', 'D', 'D', 'D'], ['C', 'C', 'D', 'C', 'D'], ['C', 'D', 'D', 'C', 'C'], ['C', 'C', 'C', 'D', 'D'], ['D', 'C', 'C', 'C', 'C']] 

Step 3 - action: west
Bumper Sensors: {'north': False, 'south': True, 'west': True, 'east': True}
Current Location: 1 , 0
Current Environment: [['C', 'C', 'D

## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents. 
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb))

In [318]:
# Your code goes here
import csv

dimensions = [5, 10, 100]
# Number of dirty squares are the dimensions of one side + 2
# Number of steps is the total area of the room
with open('test_simulation_study.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Size", "Trial", "Randomized Agent", "Simple Reflex Agent", "Model-Based Reflex Agent"])
    
    for size in dimensions:
        sizeLabel = str(size) + "x" + str(size)
        for i in range(5):
            randomX = random.randint(0, 4)
            randomY = random.randint(0, 4)
            randomOut = simple_environment(simple_randomized_agent, size, size + 2, size**2, (randomX,randomY), True)
            reflexOut = simple_environment(simple_reflex_agent, size, size + 2, size**2, (randomX,randomY), False)
            modelOut = simple_environment(agent.act, size, size + 2, size**2, (randomX,randomY), False)
            writer.writerow([sizeLabel, i, randomOut, reflexOut, modelOut])

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | | | |
| 10x10   | | | |
| 100x100 | | | |

Add charts to compare the performance of the different agents.

In [319]:
# Your graphs and discussion of the results goes here

## Task 5: Robustness of the agent implementations [10 Points] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

In [320]:
# Answer goes here

## Graduate student advanced task: Obstacles [10 Points]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus Points].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [321]:
# Your code and discussion goes here

## More advanced implementation tasks

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actualy depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currebntly clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [322]:
# Your ideas/code